# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-a7db882dd9-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Literary Movements and Divine Comedy explorative search") 

Consider the following exploratory information need:

> investigate the literary movements and the poets. In particular, compare some italian poets in terms of place of birth, number of literary works and education.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q40185`| The Divine Comedy  | node |





Also consider

```
wd:Q40185 ?p ?obj .
```

is the BGP to retrieve all **properties of The Divine Comedy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Who wrote The Divine Comedy? (return the IRI and the label of the author)

2. Identify the BGP for poet

3. Identify the BGP for literary movement

4. Return the poets born in the same place of the author of The Divine Comedy who are born between 1300 and 1500 (1300 and 1500 included) (the result set must be a list of couples poets IRI and label).

5. Identify the BGP for literary work

6. Compare the author of The Divine Comedy with Giovanni Boccaccio. Return for these poets the number of literary work they wrote (the result set must be two triples -one for each author- author IRI, label and #literary work).

7. Consider italian poets, their place of birth (the city) and their university education. Return for each city the number of poets who born there and who was educated there (the result set must be a list of triples city IRI, label and #poets).

8. Return the italian poets who won the Nobel Prize in Literature (the result set must be poet IRI and label). 

## Task 1
Who wrote The Divine Comedy? (return the IRI and the label of the author)

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q40185 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2551'), ('name', 'used metre')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de Fran

Final query for this task

In [3]:
# write your final query
queryString = """
SELECT DISTINCT ?auteur ?name
WHERE {
   # bind something
   wd:Q40185 wdt:P50 ?auteur .
   # get the label
   ?auteur sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('auteur', 'http://www.wikidata.org/entity/Q1067'), ('name', 'Dante Alighieri')]
1


## Task 2
Identify the BGP for poet

In [4]:
# write your queries


Final query for this task

In [22]:
# write your final query
# we're retrieveing all the occupations of Dante
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q1067 wdt:P106 ?o . 
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q12144794'), ('name', 'prosaist')]
[('o', 'http://www.wikidata.org/entity/Q15994177'), ('name', 'political theorist')]
[('o', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('o', 'http://www.wikidata.org/entity/Q482980'), ('name', 'author')]
[('o', 'http://www.wikidata.org/entity/Q4964182'), ('name', 'philosopher')]
[('o', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('o', 'http://www.wikidata.org/entity/Q822146'), ('name', 'lyricist')]
[('o', 'http://www.wikidata.org/entity/Q82955'), ('name', 'politician')]
[('o', 'http://www.wikidata.org/entity/Q58968'), ('name', 'intellectual')]
9


## Task 3
Identify the BGP for literary movement

In [48]:
# write your queries
# we're retrieving all the properties and object of poet
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
    wd:Q49757 ?p ?obj .
   # get the label
   ?p sc:name ?name .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org

In [49]:
# we're now trying with the properties of author
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
    wd:Q482980 ?p ?obj .
   # get the label
   ?p sc:name ?name .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org

In [5]:
#trying with the properties of Dante?
# we didn't find literary movement but we find movement
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1067 ?p ?o . 
   # get the label
   ?p sc:name ?name .
   FILTER(!isLiteral(?o)) .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1299'), ('name', 'depicted by')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('name', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('p', 'http://www.wikid

In [13]:
# let's take a look at what it is for Dante
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q1067 wdt:P135 ?o . 
   # get the label
   ?o sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q542036'), ('name', 'Dolce Stil Novo')]
1


Final query for this task

In [12]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?namep ?o ?name
WHERE {
   # bind something
   wd:Q542036 ?p ?o . 
   # get the label
   ?p sc:name ?namep .
   ?o sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('namep', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('namep', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q3326717'), ('name', 'literary movement')]
2


## Task 4
Return the poets born in the same place of the author of The Divine Comedy who are born between 1300 and 1500 (1300 and 1500 included) (the result set must be a list of couples poets IRI and label).

In [9]:
# write your queries
#We want to find birthplace and birthdate retrieving the property of Dante
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q1067 ?p ?obj .
   # get the label
   ?p sc:name ?name .
   FILTER REGEX (?name, ".*birth.*")
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
2


In [10]:
#We're decomposing the final query in 2 to be sure.
queryString = """
SELECT DISTINCT ?poet ?name ?year
WHERE {
   # bind something
   ?poet wdt:P106 wd:Q49757 ; 
       sc:name ?name ;   
       wdt:P569 ?year .
   
   FILTER(?year > "1299"^^xsd:gYear && ?year < "1501"^^xsd:gYear)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q3296224'), ('name', 'Muhammad ibn Mahmud Amuli'), ('year', '1300-01-01T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q18214823'), ('name', 'Georges Warda'), ('year', '1300-01-01T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q1510346'), ('name', 'Gerbert de Montreuil'), ('year', '1300-01-01T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q5199'), ('name', 'Le Chastelain de Couci'), ('year', '1300-01-01T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q582581'), ('name', 'Tomaso di Sasso'), ('year', '1300-01-01T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q734140'), ('name', 'Walter of Winterburn'), ('year', '1300-01-01T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q7461783'), ('name', 'Shahin Shirazi'), ('year', '1300-01-01T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q6043494'), ('name', 'Mustafa Zarir'), ('year', '1301-01-01T00:00:00Z')]
[('poet', 'http://www.wikidata.org/ent

In [71]:
#retrieving all the poets born at the same place as Dante
queryString = """
SELECT DISTINCT ?poet ?name ?birthplace ?city
WHERE {
   # bind something
   ?poet wdt:P106 wd:Q49757 ; 
       sc:name ?name ;   
       wdt:P19 ?birthplace .
    ?birthplace sc:name ?city .
   wd:Q1067 wdt:P19 ?birthplaceD . #retrieve the birthplace of Dante
   
   FILTER(?birthplace = ?birthplaceD)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q17203201'), ('name', 'Sebastiano Fenzi'), ('birthplace', 'http://www.wikidata.org/entity/Q2044'), ('city', 'Florence')]
[('poet', 'http://www.wikidata.org/entity/Q21003797'), ('name', 'Michel Verin'), ('birthplace', 'http://www.wikidata.org/entity/Q2044'), ('city', 'Florence')]
[('poet', 'http://www.wikidata.org/entity/Q426305'), ('name', 'Giovanni Maria Cecchi'), ('birthplace', 'http://www.wikidata.org/entity/Q2044'), ('city', 'Florence')]
[('poet', 'http://www.wikidata.org/entity/Q1248954'), ('name', 'Giovanni di Bernardo Rucellai'), ('birthplace', 'http://www.wikidata.org/entity/Q2044'), ('city', 'Florence')]
[('poet', 'http://www.wikidata.org/entity/Q7803'), ('name', 'Bronzino'), ('birthplace', 'http://www.wikidata.org/entity/Q2044'), ('city', 'Florence')]
[('poet', 'http://www.wikidata.org/entity/Q1067'), ('name', 'Dante Alighieri'), ('birthplace', 'http://www.wikidata.org/entity/Q2044'), ('city', 'Florence')]
[('poet', 'http://ww

Final query for this task

In [12]:
# write your final query
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {
   # bind something
   ?poet wdt:P106 wd:Q49757 ; 
       sc:name ?name ;   
       wdt:P569 ?year ;
       wdt:P19 ?birthplace .
   wd:Q1067 wdt:P19 ?birthplaceD . #retrieve the birthplace of Dante
   
   FILTER(?year > "1299"^^xsd:gYear && ?year < "1501"^^xsd:gYear && ?birthplace = ?birthplaceD)
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q21003797'), ('name', 'Michel Verin')]
[('poet', 'http://www.wikidata.org/entity/Q1248954'), ('name', 'Giovanni di Bernardo Rucellai')]
[('poet', 'http://www.wikidata.org/entity/Q177854'), ('name', "Lorenzo de' Medici")]
[('poet', 'http://www.wikidata.org/entity/Q328117'), ('name', 'Girolamo Benivieni')]
[('poet', 'http://www.wikidata.org/entity/Q721846'), ('name', 'Luigi Alamanni')]
[('poet', 'http://www.wikidata.org/entity/Q1399'), ('name', 'Niccolò Machiavelli')]
[('poet', 'http://www.wikidata.org/entity/Q366499'), ('name', 'Orcagna')]
[('poet', 'http://www.wikidata.org/entity/Q1402'), ('name', 'Giovanni Boccaccio')]
[('poet', 'http://www.wikidata.org/entity/Q631635'), ('name', 'Luigi Pulci')]
[('poet', 'http://www.wikidata.org/entity/Q2857170'), ('name', 'Antonio Alamanni')]
[('poet', 'http://www.wikidata.org/entity/Q1527917'), ('name', 'Giuliano Dati')]
[('poet', 'http://www.wikidata.org/entity/Q45539848'), ('name', 'Niccolò Tinucc

## Task 5
Identify the BGP for literary work

In [40]:
# write your queries
# looking for the objects of which The Divine Comedy is an instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q40185 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q47461344'), ('name', 'written work')]
1


Final query for this task

In [37]:
# write your final query
# we're retrieving all properties and objects of written work
queryString = """
SELECT DISTINCT ?p ?name ?obj ?nameo
WHERE {
   # bind something
   wd:Q47461344 ?p ?obj .
   # get the label
   ?p sc:name ?name .
   ?obj sc:name ?nameo .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P214'), ('nameo', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P407'), ('nameo', 'language of work or name')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q17537576'), ('nameo', 'creative work')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q7725634'), ('nameo', 'literary work')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q3331189'), ('nameo', 'version, edition, or translation')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q571'), ('nameo', 'book')]
[('p',

## Task 6
Compare the author of The Divine Comedy with Giovanni Boccaccio. Return for these poets the number of literary work they wrote (the result set must be two triples -one for each author- author IRI, label and #literary work).

In [44]:
# write your queries
#we're gonna retrieve what property link literary work and poet
queryString = """
SELECT DISTINCT  ?p ?name 
WHERE { 
   # bind something
   ?poet wdt:P106 wd:Q49757;
         ?p wd:Q7725634 .
   ?p sc:name ?name .
} 
LIMIT 50 
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
1


In [46]:
#maybe the other way around
queryString = """
SELECT DISTINCT  ?p ?name 
WHERE { 
   # bind something
   ?litwrk wdt:P31 wd:Q7725634 ;
           ?p wd:Q49757 .
   ?p sc:name ?name .
} 
LIMIT 50 
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]
3


In [50]:
#not finding anything, we're trying to retrieve every property from literary work
#and we find author(and creator, don't know what is the difference between both)
queryString = """
SELECT DISTINCT  ?p ?name 
WHERE { 
   # bind something
   ?litwrk wdt:P31 wd:Q7725634 ;
           ?p ?o .
   ?p sc:name ?name .
   FILTER(!isLiteral(?o)) .
} 
LIMIT 40 
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P110'), ('name', 'illustrator')]
[('p', 'http://www.wikidata.org/prop/direct/P123'), ('name', 'publisher')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('name', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1434'), ('name', 'takes place in fictional universe')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direc

Final query for this task

In [52]:
# write your final query
# we found the BGP of Giovanni Boccaccio during the task 4
queryString = """
SELECT DISTINCT ?poet ?name (COUNT(?lwork) as ?nblworks) 
WHERE{ 
   # bind something
   ?lwork wdt:P31 wd:Q7725634 ;
          wdt:P50 ?poet .
   ?poet wdt:P106 wd:Q49757 ;
        sc:name ?name .
   FILTER (?poet IN (wd:Q1402, wd:Q1067)) .
} ORDER BY DESC(?nblworks)
LIMIT 50 
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q1067'), ('name', 'Dante Alighieri'), ('nblworks', '37')]
[('poet', 'http://www.wikidata.org/entity/Q1402'), ('name', 'Giovanni Boccaccio'), ('nblworks', '10')]
2


## Task 7
Consider italian poets, their place of birth (the city) and their university education. Return for each city the number of poets who born there and who was educated there (the result set must be a list of triples city IRI, label and #poets).

In [14]:
# write your queries
# First we need to find the BGP of Italian nationality. We just need to retrieve the nationality of Dante
# It was not Italy at the time
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q1067 wdt:P27 ?o . 
   # get the label
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q148540'), ('name', 'Republic of Florence')]
1


In [27]:
#we got the PGB of country above, we're just gonna use it on the birthplace of Dante to obtain the PGB of Italy
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
   # bind something
   wd:Q1067 wdt:P19 ?city. 
   # get the label
   ?city wdt:P17 ?country.
   ?country sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
1


In [66]:
# we also need the BGP of university education
# We're gonna try to find the BGP of city to see its subclasses
queryString = """
SELECT DISTINCT ?city ?name
WHERE {
   # bind something
   ?born wdt:P19 ?pot . 
   ?pot wdt:P31 ?type .
   ?type wdt:P279 ?city .
   ?city sc:name ?name .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q12081657'), ('name', 'township level division of the Republic of China')]
[('city', 'http://www.wikidata.org/entity/Q14502790'), ('name', 'settlement of Hungary')]
[('city', 'http://www.wikidata.org/entity/Q22580836'), ('name', 'village in Latvia')]
[('city', 'http://www.wikidata.org/entity/Q717478'), ('name', 'administrative territorial entity of the United Kingdom')]
[('city', 'http://www.wikidata.org/entity/Q683738'), ('name', 'Administrative subdivision of Wales')]
[('city', 'http://www.wikidata.org/entity/Q2828312'), ('name', 'protected area of Canada')]
[('city', 'http://www.wikidata.org/entity/Q10677154'), ('name', 'Stadsdelsnämndsområde')]
[('city', 'http://www.wikidata.org/entity/Q35798'), ('name', 'executive branch')]
[('city', 'http://www.wikidata.org/entity/Q57058'), ('name', 'municipalities of Croatia')]
[('city', 'http://www.wikidata.org/entity/Q15989253'), ('name', 'part')]
[('city', 'http://www.wikidata.org/entity/Q1995

In [18]:
# not what I'm looking for.
queryString = """
SELECT DISTINCT ?city ?name
WHERE {
   # bind something
   ?city wdt:P279 wd:Q515 ;
       sc:name ?name .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q15260353'), ('name', 'host city')]
[('city', 'http://www.wikidata.org/entity/Q5124027'), ('name', 'city proper')]
[('city', 'http://www.wikidata.org/entity/Q1656724'), ('name', 'ideal city')]
[('city', 'http://www.wikidata.org/entity/Q83020'), ('name', 'provincial city')]
[('city', 'http://www.wikidata.org/entity/Q28659128'), ('name', 'city of North Korea')]
[('city', 'http://www.wikidata.org/entity/Q63208555'), ('name', 'city in Gabon')]
[('city', 'http://www.wikidata.org/entity/Q104157'), ('name', 'city of the Philippines')]
[('city', 'http://www.wikidata.org/entity/Q23640644'), ('name', 'Levitical city')]
[('city', 'http://www.wikidata.org/entity/Q13218690'), ('name', 'town in Hungary')]
[('city', 'http://www.wikidata.org/entity/Q15141632'), ('name', 'Thesaban Nakhon')]
[('city', 'http://www.wikidata.org/entity/Q37800986'), ('name', 'Serbian city')]
[('city', 'http://www.wikidata.org/entity/Q16858213'), ('name', 'town in Romania')]


In [72]:
# Another approach could be to see the relations between Dante and Florence(as we have the BGP of both) hoping Dante has studied is Florence.
# Apparently not.
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1067 ?p wd:Q2044 .
   ?p sc:name ?name .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('name', 'residence')]
2


In [39]:
#maybe Dante doesn't have the field city of education but Giovanni Boccaccio does?
#Apparently not
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1402 ?p ?o . 
   # get the label
   ?p sc:name ?name .
   FILTER(!isLiteral(?o)) .
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('name', 'ethnic group')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('name', 'place of death')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender')]
[('p', 'http://ww

In [47]:
#Maybe the property student of can help us
#Seems to give the name of his teacher maybe this will bring us to a university
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q1402 wdt:P1066 ?o . 
   # get the label
   ?o sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q736348'), ('name', 'Cino da Pistoia')]
1


In [48]:
#we've obtained not only some universities but also the property educated at
queryString = """
SELECT DISTINCT ?p ?namep ?o ?name
WHERE {
   # bind something
   wd:Q736348 ?p ?o . 
   # get the label
   ?p sc:name ?namep .
   ?o sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('namep', 'employer'), ('o', 'http://www.wikidata.org/entity/Q1245999'), ('name', 'University of Siena')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('namep', 'employer'), ('o', 'http://www.wikidata.org/entity/Q748085'), ('name', 'University of Perugia')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('namep', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q1209498'), ('name', 'poet lawyer')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('namep', 'employer'), ('o', 'http://www.wikidata.org/entity/Q131262'), ('name', 'University of Bologna')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('namep', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q131262'), ('name', 'University of Bologna')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('namep', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q1622272'), ('name', 'university teacher')]
[('p', 'http://www.wikidata.org/prop/direct/

In [24]:
#we're looking at the University of Siena to find the PGB of university and see how it is linked to city
#we're retrieving the property located in the administrative territorial entity
queryString = """
SELECT DISTINCT ?p ?namep ?o ?name
WHERE {
   # bind something
   wd:Q1245999 ?p ?o . 
   # get the label
   ?p sc:name ?namep .
   ?o sc:name ?name .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('namep', 'member of'), ('o', 'http://www.wikidata.org/entity/Q19861084'), ('name', 'ORCID, Inc.')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('namep', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q2751'), ('name', 'Siena')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('namep', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q2751'), ('name', 'Siena')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('namep', 'country'), ('o', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('namep', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q3918'), ('name', 'university')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('namep', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q45400320'), ('name', 'open-access publisher')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('na

In [26]:
#query without the city of education is working
queryString = """
SELECT DISTINCT ?birthplace ?city (COUNT(?poet) as ?nbpoets)
WHERE {
   # bind something
   ?poet wdt:P106 wd:Q49757 ; #a poet
        sc:name ?name ;
        wdt:P19 ?birthplace .
   ?birthplace sc:name ?city ;
           wdt:P17 wd:Q38 .
} ORDER BY DESC(?nbpoets)
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('birthplace', 'http://www.wikidata.org/entity/Q220'), ('city', 'Rome'), ('nbpoets', '213')]
[('birthplace', 'http://www.wikidata.org/entity/Q2044'), ('city', 'Florence'), ('nbpoets', '179')]
[('birthplace', 'http://www.wikidata.org/entity/Q2634'), ('city', 'Naples'), ('nbpoets', '136')]
[('birthplace', 'http://www.wikidata.org/entity/Q490'), ('city', 'Milan'), ('nbpoets', '131')]
[('birthplace', 'http://www.wikidata.org/entity/Q641'), ('city', 'Venice'), ('nbpoets', '93')]
[('birthplace', 'http://www.wikidata.org/entity/Q1449'), ('city', 'Genoa'), ('nbpoets', '72')]
[('birthplace', 'http://www.wikidata.org/entity/Q495'), ('city', 'Turin'), ('nbpoets', '60')]
[('birthplace', 'http://www.wikidata.org/entity/Q546'), ('city', 'Trieste'), ('nbpoets', '59')]
[('birthplace', 'http://www.wikidata.org/entity/Q1891'), ('city', 'Bologna'), ('nbpoets', '57')]
[('birthplace', 'http://www.wikidata.org/entity/Q617'), ('city', 'Padua'), ('nbpoets', '49')]
[('birthplace', 'http://www.wikidata

In [11]:
queryString = """
SELECT DISTINCT ?poet ?city ?educ
WHERE {
   # bind something
   ?poet wdt:P106 wd:Q49757 ; #a poet
        wdt:P19 ?birthplace ;
        wdt:P69 ?university .
   ?birthplace sc:name ?city ;
        wdt:P17 wd:Q38 .
   ?university wdt:P131 ?educationcity .
   ?educationcity sc:name ?educ .
   FILTER (?birthplace = ?educationcity)
} 
LIMIT 50 
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q25106'), ('city', 'Genoa'), ('educ', 'Genoa')]
[('poet', 'http://www.wikidata.org/entity/Q318309'), ('city', 'Bologna'), ('educ', 'Bologna')]
[('poet', 'http://www.wikidata.org/entity/Q675583'), ('city', 'Rome'), ('educ', 'Rome')]
[('poet', 'http://www.wikidata.org/entity/Q273334'), ('city', 'Naples'), ('educ', 'Naples')]
[('poet', 'http://www.wikidata.org/entity/Q1130258'), ('city', 'Parma'), ('educ', 'Parma')]
[('poet', 'http://www.wikidata.org/entity/Q535810'), ('city', 'Turin'), ('educ', 'Turin')]
[('poet', 'http://www.wikidata.org/entity/Q659749'), ('city', 'Rome'), ('educ', 'Rome')]
[('poet', 'http://www.wikidata.org/entity/Q153670'), ('city', 'Turin'), ('educ', 'Turin')]
[('poet', 'http://www.wikidata.org/entity/Q733342'), ('city', 'Padua'), ('educ', 'Padua')]
[('poet', 'http://www.wikidata.org/entity/Q919900'), ('city', 'Padua'), ('educ', 'Padua')]
[('poet', 'http://www.wikidata.org/entity/Q350103'), ('city', 'Genoa'), ('educ',

Final query for this task

In [15]:
# write your final query
queryString = """
SELECT DISTINCT ?city2 (COUNT(?poet2) as ?nbpoets) WHERE {
    ?poet2 wdt:P106 wd:Q49757 ; #a poet
            wdt:P19 ?birthplace2 .
    ?birthplace2 sc:name ?city2 .       
    
    {
        SELECT DISTINCT ?poet ?city
        WHERE {
           # bind something
           ?poet wdt:P106 wd:Q49757 ; #a poet
                wdt:P19 ?birthplace ;
                wdt:P69 ?university .
           ?birthplace sc:name ?city ;
                wdt:P17 wd:Q38 .
           ?university wdt:P131 ?educationcity .
           FILTER (?birthplace = ?educationcity)
        } 
    }
    FILTER (?poet2 = ?poet)
} GROUP BY ?city2
ORDER BY DESC(?nbpoets)
LIMIT 20 
"""

print("Results")
x=run_query(queryString)

Results
[('city2', 'Rome'), ('nbpoets', '16')]
[('city2', 'Milan'), ('nbpoets', '14')]
[('city2', 'Florence'), ('nbpoets', '9')]
[('city2', 'Turin'), ('nbpoets', '9')]
[('city2', 'Padua'), ('nbpoets', '4')]
[('city2', 'Bologna'), ('nbpoets', '4')]
[('city2', 'Genoa'), ('nbpoets', '3')]
[('city2', 'Trieste'), ('nbpoets', '2')]
[('city2', 'Parma'), ('nbpoets', '2')]
[('city2', 'Naples'), ('nbpoets', '2')]
[('city2', 'Urbino'), ('nbpoets', '2')]
[('city2', 'Ferrara'), ('nbpoets', '1')]
[('city2', 'Terlizzi'), ('nbpoets', '1')]
[('city2', 'Catania'), ('nbpoets', '1')]
[('city2', 'Bari'), ('nbpoets', '1')]
[('city2', 'Siena'), ('nbpoets', '1')]
[('city2', 'Sassari'), ('nbpoets', '1')]
[('city2', 'Messina'), ('nbpoets', '1')]
[('city2', 'Chieti'), ('nbpoets', '1')]
[('city2', 'Prato'), ('nbpoets', '1')]
20


## Task 8
Return the italian poets who won the Nobel Prize in Literature (the result set must be poet IRI and label).

In [10]:
# write your queries
# As the Nobel Prize of Literature is something given to writers(I think) we're retrieving the properties of writer
# results not very relevant
queryString = """
SELECT DISTINCT ?p ?namep ?o ?name  
WHERE {
   # bind something
   wd:Q36180 ?p ?o .
    ?p sc:name ?namep .
   ?o sc:name ?name .
} 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('namep', 'field of this occupation'), ('o', 'http://www.wikidata.org/entity/Q37260'), ('name', 'writing')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('namep', 'field of this occupation'), ('o', 'http://www.wikidata.org/entity/Q2250012'), ('name', 'writing')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('namep', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q5849863'), ('name', 'Category:Writers')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('namep', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P50'), ('name', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('namep', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q12737077'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('namep', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), 

In [9]:
# we have a property Significant Person, maybe we can use it. First we'll try to see what there is in it
queryString = """
SELECT DISTINCT ?s ?nameS ?o ?name  
WHERE {
   # bind something
   ?s wdt:P3342 ?o .
    ?s sc:name ?nameS .
   ?o sc:name ?name .
} 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q15734590'), ('nameS', 'Alma Sundquist'), ('o', 'http://www.wikidata.org/entity/Q15734554'), ('name', 'Thora Wigardh')]
[('s', 'http://www.wikidata.org/entity/Q4969542'), ('nameS', 'Ada Nilsson'), ('o', 'http://www.wikidata.org/entity/Q15734554'), ('name', 'Thora Wigardh')]
[('s', 'http://www.wikidata.org/entity/Q4767613'), ('nameS', 'Anna Whitlock'), ('o', 'http://www.wikidata.org/entity/Q15734554'), ('name', 'Thora Wigardh')]
[('s', 'http://www.wikidata.org/entity/Q4990100'), ('nameS', 'Lydia Wahlström'), ('o', 'http://www.wikidata.org/entity/Q15734554'), ('name', 'Thora Wigardh')]
[('s', 'http://www.wikidata.org/entity/Q106903887'), ('nameS', 'Petroccelli'), ('o', 'http://www.wikidata.org/entity/Q16059906'), ('name', 'Vincenzo Petroccelli')]
[('s', 'http://www.wikidata.org/entity/Q485664'), ('nameS', 'Sukjong of Joseon'), ('o', 'http://www.wikidata.org/entity/Q16184469'), ('name', 'Nam Gu-man')]
[('s', 'http://www.wikidata.org/entity/Q4

In [22]:
# It's strange. For Keynes for example, it seems that the object of Significant person is a person and the subject, the reason why. But it doesn't seem the case most of the time.
# We'll try to put the object as a writer, we'll see what we can obtain
queryString = """
SELECT DISTINCT ?s ?nameS
WHERE {
   # bind something
   ?writer wdt:P106 wd:Q36180 .
   ?s wdt:P3342 ?writer ;
       sc:name ?nameS .
} 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q21936906'), ('nameS', 'Wismichu')]
[('s', 'http://www.wikidata.org/entity/Q468618'), ('nameS', 'Dominique Vivant')]
[('s', 'http://www.wikidata.org/entity/Q104450705'), ('nameS', 'Processo do padre António Vieira')]
[('s', 'http://www.wikidata.org/entity/Q958683'), ('nameS', 'Nick Kamen')]
[('s', 'http://www.wikidata.org/entity/Q2642427'), ('nameS', 'Albert Engström')]
[('s', 'http://www.wikidata.org/entity/Q48235'), ('nameS', 'stoicism')]
[('s', 'http://www.wikidata.org/entity/Q485009'), ('nameS', 'Yeongjo of Joseon')]
[('s', 'http://www.wikidata.org/entity/Q77079'), ('nameS', 'Jean Paul')]
[('s', 'http://www.wikidata.org/entity/Q4988547'), ('nameS', 'Ida Gawell-Blumenthal')]
[('s', 'http://www.wikidata.org/entity/Q549109'), ('nameS', 'Artur Hazelius')]
[('s', 'http://www.wikidata.org/entity/Q552288'), ('nameS', 'R. H. Barlow')]
[('s', 'http://www.wikidata.org/entity/Q57074'), ('nameS', 'Nelly Sachs')]
[('s', 'http://www.wikidata.org/ent

In [34]:
# It seems that we have also painters and economists in the list. Maybe we can reduce with poets.
queryString = """
SELECT DISTINCT ?s ?names
WHERE {
   # bind something
   ?poet wdt:P106 wd:Q49757 .
   ?s wdt:P3342 ?poet ;
       sc:name ?names .
} 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q468618'), ('names', 'Dominique Vivant')]
[('s', 'http://www.wikidata.org/entity/Q63213350'), ('names', 'Sentiero di Leonardo da Vinci')]
[('s', 'http://www.wikidata.org/entity/Q958683'), ('names', 'Nick Kamen')]
[('s', 'http://www.wikidata.org/entity/Q77079'), ('names', 'Jean Paul')]
[('s', 'http://www.wikidata.org/entity/Q108427348'), ('names', "Marie-Marguerite d'Alègre")]
[('s', 'http://www.wikidata.org/entity/Q4988547'), ('names', 'Ida Gawell-Blumenthal')]
[('s', 'http://www.wikidata.org/entity/Q552288'), ('names', 'R. H. Barlow')]
[('s', 'http://www.wikidata.org/entity/Q57074'), ('names', 'Nelly Sachs')]
[('s', 'http://www.wikidata.org/entity/Q3816'), ('names', 'George Sand')]
[('s', 'http://www.wikidata.org/entity/Q738671'), ('names', 'Val Bregaglia')]
[('s', 'http://www.wikidata.org/entity/Q55037230'), ('names', 'Stéphanie Félicité Ducrest de Saint-Aubin de Genlis')]
[('s', 'http://www.wikidata.org/entity/Q243068'), ('names', 'Anne

In [20]:
# Don't know if it's cheating but doing the NBA workflow we found the property award received by retrieving the properties of France
#So we're gonna try to find again this property with what we have there, Italy
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q38 ?p ?o .
   ?p sc:name ?name .
   FILTER (!isLiteral(?o)).
} 
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('name', 'office held by head of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1419'), ('name', 'shape')]
[('p', 'http://www.wikidata.org/prop/direct/P1464'), ('name', 'category for people born here')]
[('p', 'http://www.wikidata.org/prop/direct/P1465'), ('name', 'category for people who died here')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('name', 'contains administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www

In [28]:
#nothing, but we'll just look for the BGP of France then. For that we need the PGB of country
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   # bind something
   wd:Q38 wdt:P31 ?o .
   ?o sc:name ?name .
   FILTER (!isLiteral(?o)).
} 
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3624078'), ('name', 'sovereign state')]
[('o', 'http://www.wikidata.org/entity/Q6256'), ('name', 'country')]
2


In [30]:
queryString = """
SELECT DISTINCT ?country ?name 
WHERE {
   # bind something
   wd:Q38 wdt:P30 ?continent .
   ?country wdt:P31  wd:Q6256 ;
        wdt:P30 ?continent ;
        sc:name ?name .
} 
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q756617'), ('name', 'Danish Realm')]
[('country', 'http://www.wikidata.org/entity/Q907112'), ('name', 'Transnistria')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('name', 'Belarus')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('name', 'Norway')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('name', 'Latvia')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('name', 'Czech Republic')]
[('country', 'http://www.wikidata.org/entity/Q214'), ('name', 'Slovakia')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('name', 'Romania')]
[('country', 'http://www.wikidata.org/entity/Q219'), ('name', 'Bulgaria')]
[('country', 'http://www.wikidata.org/entity/Q222'), ('name', 'Albania')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('name', 'Croatia')]
[(

In [33]:
#and there we have the award received property
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q142 ?p ?o .
   ?p sc:name ?name .
   FILTER (!isLiteral(?o)).
} 
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('name', 'office held by head of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1325'), ('name', 'external data available at')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1464'), ('name', 'category for people born here')]
[('p', 'http://www.wikidata.org/prop/direct/P1465'), ('name', 'category for people who died here')]
[('p', 'http://www.wikidata.org/prop/direct/P1482'), ('name', 'Stack Exchange tag')]
[('p', 'http:

In [45]:
#we're looking a the prizes won by Italian poets
#Nobel prize is not there, but there is a lot of local prizes maybe reducing with those also receiving by French poets
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
   # bind something
   ?poet wdt:P106 wd:Q49757 ;
        wdt:P27 wd:Q38 ;
        wdt:P166 ?award .
   ?award sc:name ?name .
} 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q2108706'), ('name', 'Premio Campiello')]
[('award', 'http://www.wikidata.org/entity/Q18608364'), ('name', 'Commander with star of the Order of St. Sylvester')]
[('award', 'http://www.wikidata.org/entity/Q519214'), ('name', 'Nelly Sachs Prize')]
[('award', 'http://www.wikidata.org/entity/Q7650000'), ('name', 'Sutherland Trophy')]
[('award', 'http://www.wikidata.org/entity/Q1526935'), ('name', 'Viareggio Prize')]
[('award', 'http://www.wikidata.org/entity/Q788568'), ('name', 'Bagutta Prize')]
[('award', 'http://www.wikidata.org/entity/Q731542'), ('name', 'Strega Prize')]
[('award', 'http://www.wikidata.org/entity/Q14539729'), ('name', 'Commander of the Order of Merit of the Italian Republic')]
[('award', 'http://www.wikidata.org/entity/Q14539974'), ('name', 'Knight Grand Cross of the Order of Merit of the Italian Republic')]
[('award', 'http://www.wikidata.org/entity/Q154590'), ('name', 'Golden Bear')]
[('award', 'http://www.wikidata.or

In [47]:
#and there we (finally) have the Nobel Prize in Literature
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
   # bind something
   ?poet wdt:P106 wd:Q49757 ;
        wdt:P27 wd:Q38 ;
        wdt:P166 ?award .
   ?fpoet wdt:P106 wd:Q49757 ;
        wdt:P27 wd:Q142 ;
        wdt:P166 ?award2 .
   ?award sc:name ?name .
   FILTER(?award2 = ?award)
} 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q519214'), ('name', 'Nelly Sachs Prize')]
[('award', 'http://www.wikidata.org/entity/Q14539974'), ('name', 'Knight Grand Cross of the Order of Merit of the Italian Republic')]
[('award', 'http://www.wikidata.org/entity/Q55455135'), ('name', 'honorary doctorate of the Bordeaux Montaigne University')]
[('award', 'http://www.wikidata.org/entity/Q3405076'), ('name', 'Prix Médicis for foreign literature')]
[('award', 'http://www.wikidata.org/entity/Q60642820'), ('name', 'Viareggio-Versilia International Prize')]
[('award', 'http://www.wikidata.org/entity/Q3404594'), ('name', 'Prix Formentor')]
[('award', 'http://www.wikidata.org/entity/Q13452528'), ('name', 'Chevalier des Arts et des Lettres')]
[('award', 'http://www.wikidata.org/entity/Q20898906'), ('name', 'Golden Wreath')]
[('award', 'http://www.wikidata.org/entity/Q674146'), ('name', 'Leopard of Honour')]
[('award', 'http://www.wikidata.org/entity/Q10855271'), ('name', 'Knight of the Le

In [51]:
#according to wikipedia Dario Fo is not a poet so I'm verifying the result of my final query
#apparently no problem
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q765 wdt:P106 ?o .
   ?o sc:name ?name.
} 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2707485'), ('name', 'scenographer')]
[('o', 'http://www.wikidata.org/entity/Q9334029'), ('name', 'satirist')]
[('o', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
[('o', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('o', 'http://www.wikidata.org/entity/Q214917'), ('name', 'playwright')]
[('o', 'http://www.wikidata.org/entity/Q2259451'), ('name', 'stage actor')]
[('o', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('o', 'http://www.wikidata.org/entity/Q3387717'), ('name', 'theatrical director')]
[('o', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('o', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('o', 'http://www.wikidata.org/entity/Q482980'), ('name', 'author')]
[('o', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('o', 'http://www.wikidata.org/entity/Q644687'), ('name', 'illustrator')]
[('o', 'http://www.wikidata.org/e

In [54]:
#according to wikipedia there are other Italian poets who receive the Nobel Prize in Literature so I'm gonna verify the results of my final query
#We retrieve all the Italian receiving this Nobel Prize, apparently the data is just incomplete
queryString = """
SELECT DISTINCT ?italian ?name
WHERE {
   # bind something
   ?italian wdt:P27 wd:Q38 ;
       wdt:P166 wd:Q37922 ;
       sc:name ?name.
} 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('italian', 'http://www.wikidata.org/entity/Q765'), ('name', 'Dario Fo')]
[('italian', 'http://www.wikidata.org/entity/Q83038'), ('name', 'Salvatore Quasimodo')]
[('italian', 'http://www.wikidata.org/entity/Q83174'), ('name', 'Eugenio Montale')]
3


Final query for this task

In [46]:
# write your final query
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {
   # bind something
   ?poet wdt:P106 wd:Q49757 ;
        wdt:P27 wd:Q38 ;
        wdt:P166 wd:Q37922 ;
        sc:name ?name.
} 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q765'), ('name', 'Dario Fo')]
[('poet', 'http://www.wikidata.org/entity/Q83038'), ('name', 'Salvatore Quasimodo')]
[('poet', 'http://www.wikidata.org/entity/Q83174'), ('name', 'Eugenio Montale')]
3
